In [ ]:
import os
import json
import logging
from datetime import datetime
from google import genai
from dotenv import load_dotenv


load_dotenv()
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
logger = logging.getLogger(__name__)

api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY not set")

client = genai.Client(api_key=api_key)
MODEL = "gemini-2.5-flash"

def validate_input(text):
    if not text or len(text.strip()) < 10:
        raise ValueError("Input text must be at least 10 characters.")
    return text.strip()

def summarize_text(text):
    prompt = f"Summarize the following text concisely:\n{text}"
    logger.info(f"SUMMARY PROMPT: {prompt}")
    response = client.models.generate_content(model=MODEL, contents=prompt)
    logger.info(f"SUMMARY OUTPUT: {response.text}")
    return response.text.strip()

def extract_keywords(summary):
    prompt = f"Extract 5-10 keywords as comma-separated values:\n{summary}"
    logger.info(f"KEYWORD PROMPT: {prompt}")
    response = client.models.generate_content(model=MODEL, contents=prompt)
    keywords = [k.strip() for k in response.text.split(",")]
    logger.info(f"KEYWORDS OUTPUT: {keywords}")
    return keywords

def evaluate(original, summary, keywords):
    length_ratio = len(summary) / len(original)
    length_score = 10 if 0.2 <= length_ratio <= 0.4 else 6

    relevance_score = sum(
        1 for k in keywords if k.lower() in original.lower()
    ) / max(len(keywords), 1) * 10

    avg_sentence_len = sum(len(s.split()) for s in summary.split(".")) / max(len(summary.split(".")), 1)
    clarity_score = 10 if 10 <= avg_sentence_len <= 25 else 6

    return {
        "length_score": round(length_score, 2),
        "relevance_score": round(relevance_score, 2),
        "clarity_score": round(clarity_score, 2),
        "overall_score": round((length_score + relevance_score + clarity_score) / 3, 2)
    }


def save_results(text, summary, keywords, evaluation):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")

    with open(f"result_{ts}.txt", "w") as f:
        f.write(f"SUMMARY:\n{summary}\n\nKEYWORDS:\n{', '.join(keywords)}\n\nEVALUATION:\n{evaluation}")

    with open(f"result_{ts}.json", "w") as f:
        json.dump({
            "original_text": text,
            "summary": summary,
            "keywords": keywords,
            "evaluation": evaluation
        }, f, indent=2)

def main():
    try:
        text = validate_input(input("Enter text: "))
        summary = summarize_text(text)
        keywords = extract_keywords(summary)
        evaluation = evaluate(text, summary, keywords)
        save_results(text, summary, keywords, evaluation)
        print("Pipeline executed successfully.")
    except Exception as e:
        logger.error(e)
        print("Error:", e)

if __name__ == "__main__":
    main()
